### Day 13 pt 1: Test Data

In [2]:
# path 
filepath = "day13_test_data.txt"
with open(filepath) as fh:
    lines = [line.split(',') for line in fh.readlines()]

# departure time
bus_times = [x for x in lines[1]]
diff_times = [x[0] for x in enumerate(bus_times)]
bus_dict = {x[1]: x[0] for x in enumerate(bus_times)}
bus_dict

{'7': 0, '13': 1, 'x': 5, '59': 4, '31': 6, '19': 7}

#### Recommended Reading: Number Theory 

- MF mentioned looking into number theory so did a bit of reading here: http://www.its.caltech.edu/~kpilch/olympiad/NumberTheory-Complete.pdf

- In looking through `Chinese Remainder Theory` looks like it would work 

#### Chinese Remainder Theory Overview: 

- Allows us to solve things like: 

```
Problem:
- Find all integers n such that n leaves a remainder of 2 when divided by 3, a remainder of 2 when divided by 4 and a remainder of 1 when divided by 5.
```

- For this problem, we want to find the number `t` that leaves a certain remainder (lagged time) when divided by `bus time`.
- Would look like this:
    - `t` where:
        - remainder 0 when divded by 7
        - remainder of (13 - 1) when divided by 13
            - Since we need t + 1 to be a multiple of 13, this means we need a remainder of `13-1` when `t` is divided by 13. 
        - remainder of (59 - 4) when divded by 59
        - remainder of (31 - 6) when divded by 31
        - remainder of (19 - 7) when divded by 19
        
#### Basic Example of CRT:
- Source: https://brilliant.org/wiki/chinese-remainder-theorem/
- Say we want to solve the following: 
    - `X = 2 (mod 3)`
    - `X = 3 (mod 8)`
- we would whittle down 
- Begin with the largest modulus & rewrite as an equation:
    - `x = 8j + 3`
- Substitute into first congruence
    - `8j + 3 = 2` (mod 3) 
- Solve for `j`:
    - `8j + 3 - 2` (mod 3) -->  only value that works for j is 1 (11 % 3 == 2)
    - `j` = 1 (mod 3)
- We then write this congruence as an equation & substitute into equation for k: 
    - `j = 3k + 1`
    - `x = 8(3k + 1) + 3`
    - `x = 24k + 11`

In [5]:
11 % 8

3

#### Test Case - Checking Assumptions: 

**All modulo (moduli??) must share same GCD**

- For this problem, we want to find the number `t` that leaves a certain remainder (lagged time) when divided by `bus time`.
- Would look like this:
    - `t` where:
        - remainder 0 when divded by 7
        - remainder of (13 - 1) when divided by 13
            - Since we need t + 1 to be a multiple of 13, this means we need a remainder of `13-1` when `t` is divided by 13. 
        - remainder of (59 - 4) when divded by 59
        - remainder of (31 - 6) when divded by 31
        - remainder of (19 - 7) when divded by 19

In [9]:
from itertools import combinations
import math 
test_list = [7,13,59,31,19]

output = combinations(test_list,2)
for i in output:
    print(math.gcd(i[0], i[1])) # All are 1

1
1
1
1
1
1
1
1
1
1


#### Actual solve: 

- We want `t` to be equivalent to all of the mods output. 
    - `t = 0 (mod 7)`
    - `t = 13 - 1 (mod 13)`
    - `t = 59 - 4 (mod 59)`
    - `t = 31 - 6 (mod 31)`
    - `t = 19 - 7 (mod 19)`
    
- Writing It Out: 
    - Resource: http://homepages.math.uic.edu/~leon/mcs425-s08/handouts/chinese_remainder.pdf
    - Info:
        - k = 5
        - m1 = 7, a1 = 0
        - m2 = 13, a2 = 12
        - m3 = 59, a3 = 55
        - m4 = 31, a4 = 25
        - m5 = 19, a5 = 12
    - Should be a solution with modulo m: `7 * 13 * 59 * 31 * 19 = 3162341`
    
    - Step 1: Compute various equations for zs:
        - z1 = m/m1 = m2m3m4m5 = 13 * 59 * 31 * 19= 451763
        - z2 = m/m2 = m1m3m4m5 = 7 * 59 * 31 * 19 = 243257
        - z3 = m / m3 = m1m2m4m5 = 7 * 13 * 31 * 19 = 53599
        - z4 = m / m4 = m1m2m3m5 = 7 * 13 * 59 * 19 =  102011
        - z5 = m / m5 = m1m2m3m4 = 7 * 13 * 59 * 31 = 166439
   
    - Step 2: Compute inverse modulo
```python
pow(451763, -1, 7) == 2
pow(243257, -1, 13) == 1
pow(53599, -1, 59) == 35
pow(102011, -1, 31) == 3
pow(166439, -1, 19) == 18
```
    - 
  - Step 3: Solve for w
      - w1 = y1z1 (mod m) = 2 * 451763 (mod 3162341)
      - w2 = y2z2 (mod m) = 1 * 243257 (mod 3162341)
      - w3 = y3z3 (mod m) = 35 * 53599 (mod 3162341)
      - w4 = y4z4 (mod m) = 3 * 102011 (mod 3162341)        
      - w5 = y5z5 (mod m) = 18 * 166439 (mod 3162341)
  
  
  - Step 4: Throw it all together for unique solution: 
      - `x = a1*w1 + a2 * w2 + a3 * w3 + a4*w4 + a5*w5 (mod 3162341)`
      - `  = 0 * 2 * 451763 + 12 * 1 * 243257 + 55* 35 * 53599 + 25 * 3 * 102011 + 12 * 18 * 166439 mod(3162341)`
      - ` x = 1068781`


In [69]:
print((0 * 2 * 451763 + 12 * 1 * 243257 + 55* 35 * 53599 + 25 * 3 * 102011 + 12 * 18 * 166439) % 3162341)

1068781


#### Scaling to Python: 

- Now that I can solve these, let's write some basic Python code

In [82]:
from functools import reduce

m_list = [7,13,59,31,19]
a_list = [0,12,55,25,12]

# solve for modulo
modulo = reduce((lambda x, y: x * y), m_list)
print(modulo)

# Step 1: Compute various equations for zs:
# generate all zs
z_list = []
for i in range(len(m_list)):
    m_vals = [x for x in m_list if x != m_list[i]]
    z_list.append(reduce((lambda x, y: x * y), m_vals))
print(z_list)

#  Step 2: Compute inverse modulo
# calculate ys
y_list = []
for i in range(len(m_list)):
    y_list.append(pow(z_list[i], -1, m_list[i]))
print(y_list)

# Step 3: Solve for w
w_list = []
for i in range(len(m_list)):
    w_list.append(y_list[i] * z_list[i])
print(w_list)

# Step 4: Throw it Together
total_sum = 0
for i in range(len(m_list)):
    total_sum += w_list[i] * a_list[i]
print(total_sum)

# Step 5: Solve for solution by taking modulo 
print(total_sum % modulo)

3162341
[451763, 243257, 53599, 102011, 166439]
[2, 1, 35, 3, 18]
[903526, 243257, 1875965, 306033, 2995902]
149698808
1068781


### Moving to Part 2 Real Data

In [75]:
# path 
filepath = "day13_data.txt"
with open(filepath) as fh:
    lines = [line.split(',') for line in fh.readlines()]
print(lines)
# departure time
bus_times = [x for x in lines[1]]
diff_times = [x[0] for x in enumerate(bus_times)]
bus_dict = {x[1]: x[0] for x in enumerate(bus_times)}
bus_dict

[['1008169\n'], ['29', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', '41', 'x', 'x', 'x', '37', 'x', 'x', 'x', 'x', 'x', '653', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', '13', 'x', 'x', 'x', '17', 'x', 'x', 'x', 'x', 'x', '23', 'x', 'x', 'x', 'x', 'x', 'x', 'x', '823', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', '19']]


{'29': 0,
 'x': 78,
 '41': 19,
 '37': 23,
 '653': 29,
 '13': 42,
 '17': 46,
 '23': 52,
 '823': 60,
 '19': 79}

In [83]:
m_list = [29,41,37, 653, 13, 17, 23, 823, 19]
a_list = [0, 22, 14, 624, -29, -29, -29, 763, -60]

# solve for modulo
modulo = reduce((lambda x, y: x * y), m_list)
print(modulo)

# Step 1: Compute various equations for zs:
# generate all zs
z_list = []
for i in range(len(m_list)):
    m_vals = [x for x in m_list if x != m_list[i]]
    z_list.append(reduce((lambda x, y: x * y), m_vals))
print(z_list)

#  Step 2: Compute inverse modulo
# calculate ys
y_list = []
for i in range(len(m_list)):
    y_list.append(pow(z_list[i], -1, m_list[i]))
print(y_list)

# Step 3: Solve for w
w_list = []
for i in range(len(m_list)):
    w_list.append(y_list[i] * z_list[i])
print(w_list)

# Step 4: Throw it Together
total_sum = 0
for i in range(len(m_list)):
    total_sum += w_list[i] * a_list[i]
print(total_sum)

# Step 5: Solve for solution by taking modulo 
print(total_sum % modulo)

2283338533368659
[78735811495471, 55691183740699, 61711852253207, 3496689943903, 175641425643743, 134314031374627, 99275588407333, 2774408910533, 120175712282561]
[9, 36, 34, 614, 2, 1, 5, 170, 6]
[708622303459239, 2004882614665164, 2098202976609038, 2146967625556442, 351282851287486, 134314031374627, 496377942036665, 471649514790610, 721054273695366]
1701318110989628856
230903629977901
